fonte: https://www.kaggle.com/arjunprasadsarkhel/2021-olympics-in-tokyo

### Bibliotecas

In [23]:
import pandas as pd # nessáro ter instalado a biblioteca openpyxl
from google.oauth2 import service_account
from functools import reduce

### Extração

In [48]:
atletas = pd.read_excel('Athletes.xlsx')

In [25]:
atletas.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [49]:
atletas.NOC.nunique(), atletas.Discipline.nunique()

(206, 46)

In [27]:
treinadores = pd.read_excel('Coaches.xlsx')

/home/alex/.virtualenvs/etl/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [28]:
treinadores.head()

,Name,NOC,Discipline,Event
0,ABDELMAGID Wael,Egypt,Football,NaN
1,ABE Junya,Japan,Volleyball,NaN
2,ABE Katsuhiko,Japan,Basketball,NaN
3,ADAMA Cherif,Côte d'Ivoire,Football,NaN
4,AGEBA Yuya,Japan,Volleyball,NaN


In [29]:
treinadores.Event.value_counts()

Men         94
Women       88
Duet        27
Softball    20
Team        14
Baseball     6
Name: Event, dtype: int64

In [30]:
treinadores.NOC.nunique(), treinadores.Discipline.nunique()

(61, 9)

In [31]:
ent_gen = pd.read_excel('EntriesGender.xlsx')

In [32]:
ent_gen.head()

,Discipline,Female,Male,Total
0,3x3 Basketball,32,32,64
1,Archery,64,64,128
2,Artistic Gymnastics,98,98,196
3,Artistic Swimming,105,0,105
4,Athletics,969,1072,2041


In [33]:
medalhas = pd.read_excel('Medals.xlsx')

/home/alex/.virtualenvs/etl/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [34]:
medalhas.head(13)

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
2,3,Japan,27,14,17,58,5
3,4,Great Britain,22,21,22,65,4
4,5,ROC,20,28,23,71,3
5,6,Australia,17,7,22,46,6
6,7,Netherlands,10,12,14,36,9
7,8,France,10,12,11,33,10
8,9,Germany,10,11,16,37,8
9,10,Italy,10,10,20,40,7


In [35]:
medalhas['Team/NOC'].nunique()

93

In [36]:
times = pd.read_excel('Teams.xlsx')

In [37]:
times.head()

,Name,Discipline,NOC,Event
0,Belgium,3x3 Basketball,Belgium,Men
1,China,3x3 Basketball,People's Republic of China,Men
2,China,3x3 Basketball,People's Republic of China,Women
3,France,3x3 Basketball,France,Women
4,Italy,3x3 Basketball,Italy,Women


In [38]:
times.NOC.nunique(), times.Discipline.nunique()

(84, 20)

In [39]:
times.Name.value_counts()

Japan                45
United States        43
Italy                34
Germany              32
France               32
                     ..
Ludwig/Kozuch         1
Thole J./Wickler      1
Perusic/Schweiner     1
Lidy/Leila            1
Venezuela             1
Name: Name, Length: 146, dtype: int64

In [65]:
times.Event.value_counts()

Men                                 120
Women                               114
Women's Team                         40
Men's Team                           40
Mixed Team                           29
Duet                                 22
Mixed Relay                          18
Women's 4 x 200m Freestyle Relay     17
Men's 4 x 400m Relay                 16
Women's 4 x 100m Medley Relay        16
Women's 4 x 100m Freestyle Relay     16
Women's 4 x 100m Relay               16
Women's Madison                      16
Men's Madison                        16
Men's 4 x 100m Freestyle Relay       16
Mixed 4 x 100m Medley Relay          16
Mixed Doubles                        16
Men's 4 x 200m Freestyle Relay       16
Men's 4 x 100m Relay                 16
Men's 4 x 100m Medley Relay          16
Women's 4 x 400m Relay               16
4 x 400m Relay Mixed                 15
Group All-Around                     14
Team                                 10
Men's Épée Team                       9


### Transformação

#### atletas por modalidade por país

In [40]:
atletas.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [50]:
# renomeando atletas

atletas.rename(columns={'Name': 'Nome',
                        'NOC': 'Nacionalidade',
                        'Discipline': 'Disciplina'},inplace=True)

In [51]:
# Atletas com 

atletas_disc = atletas.groupby(['Nacionalidade', 'Disciplina'])['Nome'].nunique() 

In [52]:
atletas_disc

Nacionalidade  Disciplina         
Afghanistan    Athletics              2
               Shooting               1
               Swimming               1
               Taekwondo              1
Albania        Artistic Gymnastics    1
                                     ..
Zambia         Swimming               2
Zimbabwe       Athletics              1
               Golf                   1
               Rowing                 1
               Swimming               2
Name: Nome, Length: 2112, dtype: int64

In [53]:
atletas_disc = pd.DataFrame(atletas_disc).reset_index().rename(columns={'Nome': 'numero_atletas', 
                                                                        'Disciplina': 'modalidade', 
                                                                        'Nacionalidade': 'pais'})

#### Treinadores por modalidade e país

In [54]:
treinadores.head()

,Name,NOC,Discipline,Event
0,ABDELMAGID Wael,Egypt,Football,NaN
1,ABE Junya,Japan,Volleyball,NaN
2,ABE Katsuhiko,Japan,Basketball,NaN
3,ADAMA Cherif,Côte d'Ivoire,Football,NaN
4,AGEBA Yuya,Japan,Volleyball,NaN


In [55]:
treinadores_disc = pd.DataFrame(treinadores.groupby(['NOC', 'Discipline'])['Name'].nunique()).\
reset_index().rename(columns={'Name': 'numero_treinadores', 
                              'Discipline': 'modalidade', 
                              'NOC': 'pais'})

In [56]:
treinadores_disc

,pais,modalidade,numero_treinadores
0,Angola,Handball,1
1,Argentina,Artistic Swimming,1
2,Argentina,Basketball,1
3,Argentina,Football,1
4,Argentina,Handball,1
...,...,...,...
156,United States of America,Volleyball,5
157,United States of America,Water Polo,2
158,Venezuela,Basketball,6
159,Venezuela,Volleyball,4


#### Times por disciplina por país

In [57]:
times.head()

,Name,Discipline,NOC,Event
0,Belgium,3x3 Basketball,Belgium,Men
1,China,3x3 Basketball,People's Republic of China,Men
2,China,3x3 Basketball,People's Republic of China,Women
3,France,3x3 Basketball,France,Women
4,Italy,3x3 Basketball,Italy,Women


In [58]:
times['categoria'] = times.Event.apply(lambda x: 'Masculino' if 'Men' in x
                                        else ('Feminino' if 'Women' in x
                                            else ('Misto' if 'Mixed' in x
                                                else 'Outros'))) 

In [59]:
times.tail()

,Name,Discipline,NOC,Event,categoria
738,South Africa,Water Polo,South Africa,Women,Feminino
739,Spain,Water Polo,Spain,Men,Masculino
740,Spain,Water Polo,Spain,Women,Feminino
741,United States,Water Polo,United States of America,Men,Masculino
742,United States,Water Polo,United States of America,Women,Feminino


In [60]:
times.categoria.value_counts()

Masculino    299
Feminino     292
Misto         94
Outros        58
Name: categoria, dtype: int64

In [61]:
times_masc = pd.DataFrame(times[times.categoria == 'Masculino'].groupby(['NOC', 'Discipline'])['Name'].nunique()).\
reset_index().rename(columns={'Name': 'times_masculinos', 
                              'Discipline': 'modalidade', 
                              'NOC': 'pais'})

In [63]:
times_fem = pd.DataFrame(times[times.categoria == 'Feminino'].groupby(['NOC', 'Discipline'])['Name'].nunique()).\
    reset_index().rename(columns={'Name': 'times_femininos',
                                'Discipline': 'modalidade',
                                'NOC': 'pais'})

In [64]:
times_misto = pd.DataFrame(times[times.categoria == 'Misto'].groupby(['NOC', 'Discipline'])['Name'].nunique()).\
reset_index().rename(columns={'Name': 'times_mistos', 
                              'Discipline': 'modalidade', 
                              'NOC': 'pais'})

In [65]:
times_outros = pd.DataFrame(times[times.categoria == 'Outros'].groupby(['NOC', 'Discipline'])['Name'].nunique()).\
reset_index().rename(columns={'Name': 'times_outros', 
                              'Discipline': 'modalidade', 
                              'NOC': 'pais'})

In [66]:
times_masc.head()

,pais,modalidade,times_masculinos
0,Argentina,Basketball,1
1,Argentina,Beach Volleyball,1
2,Argentina,Football,1
3,Argentina,Handball,1
4,Argentina,Hockey,1


In [67]:
times_fem.head()

,pais,modalidade,times_femininos
0,Angola,Handball,1
1,Argentina,Beach Volleyball,1
2,Argentina,Hockey,1
3,Argentina,Volleyball,1
4,Australia,Athletics,1


In [68]:
times_misto.head()

,pais,modalidade,times_mistos
0,Australia,Archery,1
1,Australia,Swimming,1
2,Australia,Table Tennis,1
3,Australia,Triathlon,1
4,Austria,Table Tennis,1


In [69]:
times_outros.head()

,pais,modalidade,times_outros
0,Australia,Artistic Swimming,1
1,Australia,Baseball/Softball,1
2,Australia,Rhythmic Gymnastics,1
3,Austria,Artistic Swimming,1
4,Azerbaijan,Rhythmic Gymnastics,1


#### Juntando tabelas

In [70]:
data_frames = [atletas_disc, 
               treinadores_disc, 
               times_masc, 
               times_fem, 
               times_misto, 
               times_outros]

In [71]:
df_final = reduce(lambda left, right: pd.merge(left, 
                                               right, on=['pais', 'modalidade'], 
                                               how='outer'), 
                  data_frames)

In [73]:
df_final.head()

,pais,modalidade,numero_atletas,numero_treinadores,times_masculinos,times_femininos,times_mistos,times_outros
0,Afghanistan,Athletics,2.0,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,Shooting,1.0,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,Swimming,1.0,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,Taekwondo,1.0,NaN,NaN,NaN,NaN,NaN
4,Albania,Artistic Gymnastics,1.0,NaN,NaN,NaN,NaN,NaN


In [74]:
# Substituindo Valores nullos por zero
df_final = reduce(lambda left, right: pd.merge(left, 
                                               right, on=['pais', 'modalidade'], 
                                               how='outer'), 
                  data_frames).fillna(0)

In [75]:
df_final

,pais,modalidade,numero_atletas,numero_treinadores,times_masculinos,times_femininos,times_mistos,times_outros
0,Afghanistan,Athletics,2.0,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,Shooting,1.0,0.0,0.0,0.0,0.0,0.0
2,Afghanistan,Swimming,1.0,0.0,0.0,0.0,0.0,0.0
3,Afghanistan,Taekwondo,1.0,0.0,0.0,0.0,0.0,0.0
4,Albania,Artistic Gymnastics,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2114,Dominican Republic,Basketball,0.0,4.0,0.0,0.0,0.0,0.0
2115,San Marino,Artistic Swimming,0.0,2.0,0.0,0.0,0.0,0.0
2116,Serbia,Artistic Swimming,0.0,1.0,0.0,0.0,0.0,0.0
2117,Slovakia,Artistic Swimming,0.0,1.0,0.0,0.0,0.0,0.0


In [76]:
df_final[df_final.pais == 'Brazil']

,pais,modalidade,numero_atletas,numero_treinadores,times_masculinos,times_femininos,times_mistos,times_outros
239,Brazil,Archery,2.0,0.0,0.0,0.0,1.0,0.0
240,Brazil,Artistic Gymnastics,6.0,0.0,1.0,0.0,0.0,0.0
241,Brazil,Athletics,53.0,0.0,1.0,1.0,1.0,0.0
242,Brazil,Badminton,2.0,0.0,0.0,0.0,0.0,0.0
243,Brazil,Beach Volleyball,7.0,0.0,2.0,2.0,0.0,0.0
244,Brazil,Boxing,7.0,0.0,0.0,0.0,0.0,0.0
245,Brazil,Canoe Slalom,2.0,0.0,0.0,0.0,0.0,0.0
246,Brazil,Canoe Sprint,3.0,0.0,0.0,0.0,0.0,0.0
247,Brazil,Cycling BMX Racing,2.0,0.0,0.0,0.0,0.0,0.0
248,Brazil,Cycling Mountain Bike,3.0,0.0,0.0,0.0,0.0,0.0


### Carregamento dos Dados

In [77]:
credentials = service_account.Credentials.from_service_account_file(filename='/home/alex/apps/drivers/ContaServicoBigQuery.json', 
                                                                    scopes=["https://www.googleapis.com/auth/cloud-platform"])

In [78]:
df_final.to_gbq(credentials=credentials, 
                destination_table='curso_etl.etl_excel', 
                if_exists='replace')